<a href="https://colab.research.google.com/drive/1Ib3QcE9TgUZHuYXgepwk9FS-eSbDniQ8?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

## What is Automatic Prompt Engineer (APE)?

**Automatic Prompt Engineer (APE)** is an AI based system that automatically generates and optimizes prompts for language models. Instead of manually crafting prompts through trial and error, APE uses AI to create, test and select the best prompts for your specific tasks.

### 🎯 Key Concepts:

**🔄 How APE Works:**
1. **Generate**: Creates multiple prompt candidates using different techniques
2. **Evaluate**: Scores each prompt based on clarity, effectiveness, and completeness  
3. **Optimize**: Selects the best-performing prompt for your task
4. **Test**: Runs the optimal prompt to produce your desired output

**🧠 Core Benefits:**
- **Time-Saving**: Eliminates hours of manual prompt tweaking
- **Quality**: Often produces better prompts than human engineers
- **Consistency**: Provides reliable, repeatable results
- **Learning**: Shows you different prompting approaches and techniques

**🔬 Advanced Techniques Included:**
- **Meta-prompting**: AI creates prompts for other AI systems
- **Chain-of-thought**: Step-by-step reasoning optimization
- **Persona-based**: Expert role assignments for specialized tasks
- **Multi-dimensional evaluation**: Comprehensive prompt quality assessment
- **Temperature optimization**: Automatic creativity/precision balancing

### 📊 Real-World Impact:
- Up to 40% improvement in AI response accuracy
- Better task completion rates across diverse applications


In [2]:
!pip install -q -U langchain-groq>=0.2.4

print("✅ Dependencies installed!")

✅ Dependencies installed!


In [3]:
import os
import re
import getpass
from typing import List, Dict, Tuple
from dataclasses import dataclass
from IPython.display import display, Markdown

from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

### 🔑 Provide Groq API Key

- [Groq API Key](https://console.groq.com/keys)


In [4]:
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter Groq API Key: ")

print("✅ Setup complete!")

Enter Groq API Key: ··········
✅ Setup complete!


In [6]:
@dataclass
class SimplePrompt:
    """Simple prompt container"""
    text: str
    score: float = 0.0
    technique: str = ""
    temperature: float = 0.5

    def show(self):
        """Display the prompt nicely"""
        display(Markdown(f"""
        **Technique:** {self.technique} | **Score:** {self.score:.1f}/10 | **Temperature:** {self.temperature}

        ```
        {self.text}
        ```
        """))

In [14]:
class APE:
    """Automatic Prompt Engineer"""

    def __init__(self):
        self.llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.5)
        print("APE ready!")

    def generate_prompts(self, task: str) -> List[SimplePrompt]:
        """Generate different types of optimized prompts"""
        print("🔄 Generating optimized prompts...")

        # Single comprehensive prompt to generate multiple types
        generator = ChatPromptTemplate.from_messages([
            ("system", """You are an expert prompt engineer. Create 5 different high-quality prompts for the given task using these techniques:

            1. ZERO_SHOT: Direct instruction with clear expectations
            2. EXPERT_PERSONA: Assign relevant expertise and professional approach
            3. STEP_BY_STEP: Chain-of-thought with structured reasoning
            4. EXAMPLE_BASED: Few-shot with examples and patterns
            5. COMPREHENSIVE: Detailed instructions with verification

            Make each prompt effective and unique."""),

            ("human", """Task: {task}

            Create 5 optimized prompts using the techniques above. Format exactly as:

            ZERO_SHOT:
            [prompt text]

            EXPERT_PERSONA:
            [prompt text]

            STEP_BY_STEP:
            [prompt text]

            EXAMPLE_BASED:
            [prompt text]

            COMPREHENSIVE:
            [prompt text]""")
        ])

        result = self.llm.invoke(generator.format_prompt(task=task))
        return self._parse_prompts(result.content)

    def _parse_prompts(self, response: str) -> List[SimplePrompt]:
        """Parse generated prompts"""
        prompts = []
        techniques = ["ZERO_SHOT", "EXPERT_PERSONA", "STEP_BY_STEP", "EXAMPLE_BASED", "COMPREHENSIVE"]
        temperatures = [0.3, 0.4, 0.2, 0.5, 0.3]  # Optimal temps for each technique

        for i, technique in enumerate(techniques):
            pattern = f"{technique}:\\s*(.*?)(?={techniques[i+1] if i+1 < len(techniques) else '$'})"
            match = re.search(pattern, response, re.DOTALL | re.IGNORECASE)

            if match:
                prompt_text = match.group(1).strip()
                prompt = SimplePrompt(
                    text=prompt_text,
                    technique=technique.lower().replace('_', '-'),
                    temperature=temperatures[i]
                )
                prompts.append(prompt)

        print(f"✅ Generated {len(prompts)} prompts")
        return prompts

    def evaluate_prompts(self, task: str, prompts: List[SimplePrompt]) -> List[SimplePrompt]:
        """Evaluate and score prompts"""
        print("📊 Evaluating prompts...")

        evaluator = ChatPromptTemplate.from_messages([
            ("system", """You are a prompt evaluation expert. Score prompts 1-10 based on:
            - Clarity and specificity
            - Likelihood of good results
            - Completeness for the task
            - Technical effectiveness

            Be strict but fair in scoring."""),

            ("human", """Task: {task}

            Evaluate this prompt and give a single score 1-10:

            Technique: {technique}
            Prompt: {prompt_text}

            Response format: Score: [number]/10""")
        ])

        for prompt in prompts:
            result = self.llm.invoke(evaluator.format_prompt(
                task=task,
                technique=prompt.technique,
                prompt_text=prompt.text
            ))

            # Extract score
            score_match = re.search(r"Score:\s*(\d+(?:\.\d+)?)/10", result.content)
            if score_match:
                prompt.score = float(score_match.group(1))
            else:
                prompt.score = 5.0  # Default score

        # Sort by score
        sorted_prompts = sorted(prompts, key=lambda x: x.score, reverse=True)
        print(f"✅ Evaluation complete! Best score: {sorted_prompts[0].score:.1f}/10")

        return sorted_prompts

    def test_best_prompt(self, task: str, best_prompt: SimplePrompt) -> str:
        """Test the best prompt and generate result"""
        print("🧪 Testing best prompt...")

        # Use the best prompt with its optimal temperature
        test_llm = ChatGroq(model="llama-3.1-8b-instant", temperature=best_prompt.temperature)

        tester = ChatPromptTemplate.from_messages([
            ("system", "You are an AI assistant following the given prompt instructions carefully."),
            ("human", "{prompt}\n\nTask: {task}")
        ])

        result = test_llm.invoke(tester.format_prompt(prompt=best_prompt.text, task=task))
        return result.content

    def run_ape(self, task: str) -> Tuple[SimplePrompt, str]:
        """Run the complete APE process"""
        print("🚀" + "="*50)
        print("="*52)
        print(f"📋 Task: {task}")
        print("="*52 + "\n")

        # Generate prompts
        prompts = self.generate_prompts(task)

        # Evaluate prompts
        evaluated_prompts = self.evaluate_prompts(task, prompts)

        # Get best prompt
        best_prompt = evaluated_prompts[0]

        # Test best prompt
        result = self.test_best_prompt(task, best_prompt)

        print("✅ APE process complete!\n")

        return best_prompt, result

In [12]:
def optimize_prompt(task: str):
    """
    One-function prompt optimization!

    Args:
        task (str): The task you want to optimize a prompt for

    Returns:
        Displays the best prompt and its result
    """
    ape = APE()
    best_prompt, result = ape.run_ape(task)

    # Display results nicely
    display(Markdown(f"""
    ## 🎯 Optimization Results

    **Your Task:** {task}

    ### 🏆 Best Prompt Found:
    **Technique:** {best_prompt.technique}
    **Score:** {best_prompt.score:.1f}/10
    **Temperature:** {best_prompt.temperature}

    ```
    {best_prompt.text}
    ```

    ### 📝 Generated Result:
    {result}
    """))

    return best_prompt, result

def compare_prompts(task: str):
    """
    Generate and compare multiple prompts

    Args:
        task (str): The task to generate prompts for

    Returns:
        Shows all generated prompts with scores
    """
    ape = APE()

    print("🔄 Generating and comparing prompts...\n")

    # Generate and evaluate
    prompts = ape.generate_prompts(task)
    evaluated_prompts = ape.evaluate_prompts(task, prompts)

    # Display all prompts
    display(Markdown(f"## 📊 All Generated Prompts for: {task}\n"))

    for i, prompt in enumerate(evaluated_prompts, 1):
        display(Markdown(f"### #{i} - {prompt.technique.title()} (Score: {prompt.score:.1f}/10)"))
        prompt.show()
        print()

    return evaluated_prompts

In [15]:
# Example 1: Quick optimization
print("🎯 EXAMPLE 1: Quick Prompt Optimization")
print("-" * 40)

task1 = "Write a professional email to decline a job offer politely"
best_prompt, result = optimize_prompt(task1)

🎯 EXAMPLE 1: Quick Prompt Optimization
----------------------------------------
APE ready!
🚀==================================================
📋 Task: Write a professional email to decline a job offer politely

🔄 Generating optimized prompts...
✅ Generated 5 prompts
📊 Evaluating prompts...
✅ Evaluation complete! Best score: 9.0/10
🧪 Testing best prompt...
✅ APE process complete!




    ## 🎯 Optimization Results
    
    **Your Task:** Write a professional email to decline a job offer politely
    
    ### 🏆 Best Prompt Found:
    **Technique:** zero-shot  
    **Score:** 9.0/10  
    **Temperature:** 0.3
    
    ```
    Write a polite professional email declining a job offer with a standard format and phrases.
    ```
    
    ### 📝 Generated Result:
    Subject: Declining Job Offer for [Position]

Dear [Hiring Manager's Name],

I wanted to express my sincere gratitude for extending the offer to me for the [Position] role at [Company Name]. I was truly impressed by the company culture and the team's passion for their work during my interview process.

After careful consideration, I regret to inform you that I have decided to decline the offer. Although I am excited about the opportunities that [Company Name] presents, I have decided to pursue another opportunity that aligns more closely with my long-term career goals.

Please know that my decision is not a reflection on the company or the role, but rather a personal decision that I believe is in the best interest of my career. I appreciate the time and effort that you and your team invested in the hiring process, and I am grateful for the experience.

Thank you again for the opportunity, and I wish the company continued success in the future.

Best regards,

[Your Name]

[Your Contact Information]

This email follows a standard format and includes polite phrases to decline the job offer professionally.
    

In [16]:
# Example 2: Compare different prompts
print("\n📊 EXAMPLE 2: Compare Multiple Prompts")
print("-" * 40)

task2 = "Explain quantum computing to a 12-year-old"
all_prompts = compare_prompts(task2)


📊 EXAMPLE 2: Compare Multiple Prompts
----------------------------------------
APE ready!
🔄 Generating and comparing prompts...

🔄 Generating optimized prompts...
✅ Generated 5 prompts
📊 Evaluating prompts...
✅ Evaluation complete! Best score: 9.0/10


## 📊 All Generated Prompts for: Explain quantum computing to a 12-year-old


### #1 - Zero-Shot (Score: 9.0/10)


        **Technique:** zero-shot | **Score:** 9.0/10 | **Temperature:** 0.3
        
        ```
        Explain quantum computing to a 12-year-old in 100 words or less, using simple terms and examples.
        ```
        

### #2 - Expert-Persona (Score: 9.0/10)

...


        **Technique:** step-by-step | **Score:** 9.0/10 | **Temperature:** 0.2
        
        ```
        Imagine you have a combination lock with 10 numbers. To open it, you need to try all the combinations one by one. Now, imagine a quantum computer can try all the combinations simultaneously. Explain how this works, using the concept of superposition and entanglement. Break down the process into steps, and provide an example of how a quantum computer would solve this problem.
        ```
        

### #4 - Comprehensive (Score: 9.0/10)


        **Technique:** comprehensive | **Score:** 9.0/10 | **Temperature:** 0.3
        
        ```
        Explain quantum computing to a 12-year-old, covering the following topics:

1. What is quantum computing, and how does it differ from classical computing?
2. Explain the concept of superposition and entanglement, and how they enable quantum computers to process multiple possibilities simultaneously.
3. Describe the process of quantum computing, including the use of qubits, quantum gates, and quantum algorithms.
4. Provide examples of real-world applications of quantum computing, such as cryptography and simulations.
5. Verify your explanation by answering the following questions: What are the benefits and limitations of quantum computing? How does it compare to classical computing in terms of processing power and speed?
        ```
        

### #5 - Example-Based (Score: 8.0/10)


        **Technique:** example-based | **Score:** 8.0/10 | **Temperature:** 0.5
        
        ```
        Explain quantum computing using the following examples:

- A coin can be either heads or tails. A classical computer would have to choose one or the other. A quantum computer can exist in both states at the same time.
- A quantum computer can process multiple possibilities simultaneously, like trying all the combinations of a lock at once.
- Explain how quantum computing is used in real-world applications, such as cryptography and simulations.
        ```
        

In [ ]:
# Now you can use it for your own tasks!
print("\nYOUR TURN!")
print("-" * 40)
print("Use these functions for your own tasks:")
print()
print("# For quick optimization:")
print("optimize_prompt('your task here')")
print()
print("# To compare multiple approaches:")
print("compare_prompts('your task here')")
print()

# Uncomment and modify these lines for your own tasks:
# my_task = "Your task description here"
# my_best_prompt, my_result = optimize_prompt(my_task)